In [1]:
import pandas as pd

In [9]:
df = pd.read_csv('../data/processed/casas.csv')

In [10]:
df.head()

,tamanho,ano,garagem,preco
0,159.0,2003,2,208500
1,117.0,1976,2,181500
2,166.0,2001,2,223500
3,160.0,1915,3,140000
4,204.0,2000,3,250000


In [11]:
X = df.drop('preco', axis=1)
y = df['preco'].copy()

In [14]:
X.head()

,tamanho,ano,garagem
0,159.0,2003,2
1,117.0,1976,2
2,166.0,2001,2
3,160.0,1915,3
4,204.0,2000,3


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [16]:
X_train.shape

(1022, 3)

In [17]:
X_test.shape

(438, 3)

In [56]:
import mlflow

In [58]:
mlflow.create_experiment('house-prices-eda')

'1'

# Regressao Linear

In [64]:
mlflow.start_run(experiment_id='1')

<ActiveRun: >

In [65]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [66]:
mlflow.sklearn.log_model(lr, 'lr')

In [67]:
lr_pred = lr.predict(X_test)

In [68]:
X_test.iloc[0]

tamanho      99.0
ano        1963.0
garagem       1.0
Name: 892, dtype: float64

In [69]:
list(y_test)[0]

154500

In [70]:
from sklearn.metrics import mean_squared_error, r2_score

In [71]:
import math

mse = mean_squared_error(y_test, lr_pred) # erro quadrado médio
rmse = math.sqrt(mse) # raiz qudrada erro medio
r2 = r2_score(y_test, lr_pred) # r quadrado

mlflow.log_metric('mse', mse)
mlflow.log_metric('rmse', rmse)
mlflow.log_metric('r2', r2)

In [44]:
rmse

45592.39978251848

In [45]:
r2

0.7021153642898048

In [73]:
mlflow.end_run()

# XGB Regressor

In [80]:
from xgboost import XGBRFRegressor, XGBRegressor

In [81]:
xgb_params = {
    'learning_rate': 0.2,
    'n_estimators': 50,
    'random_state': 42
}

with mlflow.start_run(experiment_id='1'):
    xgb = XGBRegressor(**xgb_params)
    xgb.fit(X_train, y_train)
    mlflow.xgboost.log_model(xgb, 'xgboost')
    
    xgb_pred = xgb.predict(X_test)
    
    mse = mean_squared_error(y_test, xgb_pred) # erro quadrado médio
    rmse = math.sqrt(mse) # raiz qudrada erro medio
    r2 = r2_score(y_test, xgb_pred)
    
    mlflow.log_metrics({'mse': mse, 'rmse': rmse, 'r2': r2})

In [51]:
xgb_pred = xgb.predict(X_test)

In [52]:
mse = mean_squared_error(y_test, xgb_pred) # erro quadrado médio
rmse = math.sqrt(mse) # raiz qudrada erro medio
r2 = r2_score(y_test, xgb_pred)

In [53]:
mse

1297231820.9346623

In [54]:
rmse

36017.10456067592

In [55]:
r2

0.8140993994382723

In [83]:
mlflow.get_experiment('1')

<Experiment: artifact_location='file:///home/kyle/Santander_Estudo/mlflow/mlflow/notebooks/mlruns/1', experiment_id='1', lifecycle_stage='active', name='house-prices-eda', tags={}>

In [84]:
mlflow.get_experiment_by_name('house-prices-eda')

<Experiment: artifact_location='file:///home/kyle/Santander_Estudo/mlflow/mlflow/notebooks/mlruns/1', experiment_id='1', lifecycle_stage='active', name='house-prices-eda', tags={}>

In [92]:
mlflow.list_run_infos('1')

[<RunInfo: artifact_uri='file:///home/kyle/Santander_Estudo/mlflow/mlflow/notebooks/mlruns/1/8607468801614a6c88667600bff39cff/artifacts', end_time=1622005130694, experiment_id='1', lifecycle_stage='active', run_id='8607468801614a6c88667600bff39cff', run_uuid='8607468801614a6c88667600bff39cff', start_time=1622005130491, status='FINISHED', user_id='kyle'>,
 <RunInfo: artifact_uri='file:///home/kyle/Santander_Estudo/mlflow/mlflow/notebooks/mlruns/1/b83b560446624b25ba739ae718ac14cb/artifacts', end_time=1622005082850, experiment_id='1', lifecycle_stage='active', run_id='b83b560446624b25ba739ae718ac14cb', run_uuid='b83b560446624b25ba739ae718ac14cb', start_time=1622005082702, status='FINISHED', user_id='kyle'>,
 <RunInfo: artifact_uri='file:///home/kyle/Santander_Estudo/mlflow/mlflow/notebooks/mlruns/1/055cb33203ff4fd98020d386b83c07bb/artifacts', end_time=1622005053452, experiment_id='1', lifecycle_stage='active', run_id='055cb33203ff4fd98020d386b83c07bb', run_uuid='055cb33203ff4fd98020d386b8

In [97]:
mlflow.get_run('8607468801614a6c88667600bff39cff')

<Run: data=<RunData: metrics={'mse': 1386727460.1346002,
 'r2': 0.8012741720529797,
 'rmse': 37238.789724353286}, params={}, tags={'mlflow.log-model.history': '[{"run_id": "8607468801614a6c88667600bff39cff", '
                             '"artifact_path": "xgboost", "utc_time_created": '
                             '"2021-05-26 04:58:50.686167", "flavors": '
                             '{"python_function": {"loader_module": '
                             '"mlflow.xgboost", "python_version": "3.8.5", '
                             '"data": "model.xgb", "env": "conda.yaml"}, '
                             '"xgboost": {"xgb_version": "1.4.2", "data": '
                             '"model.xgb"}}}]',
 'mlflow.source.name': '/home/kyle/Santander_Estudo/mlflow/.venv/lib/python3.8/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'kyle'}>, info=<RunInfo: artifact_uri='file:///home/kyle/Santander_Estudo/mlflow/mlflow/notebooks/mlruns/1/8607468801614a6c886

In [100]:
[mlflow.get_run(run.run_id).data.metrics for run in mlflow.list_run_infos('1')]

[{'mse': 1386727460.1346002,
  'r2': 0.8012741720529797,
  'rmse': 37238.789724353286},
 {'mse': 1297231820.9346623,
  'r2': 0.8140993994382723,
  'rmse': 36017.10456067592},
 {'mse': 25674268731.65471,
  'r2': -2.679266804262422,
  'rmse': 160231.9216999369},
 {'mse': 25674268731.65471,
  'r2': -2.679266804262422,
  'rmse': 160231.9216999369},
 {'mse': 25674268731.65471,
  'r2': -2.679266804262422,
  'rmse': 160231.9216999369},
 {'mse': 1297231820.9346623,
  'r2': 0.8140993994382723,
  'rmse': 36017.10456067592},
 {'mse': 2078666917.9289908,
  'r2': 0.7021153642898048,
  'rmse': 45592.39978251848}]